In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

load_dotenv()
directory = os.getenv("path")
# main_dir = os.getenv("main_dir")

In [ ]:
model = AffinityPropagation()

# Clustering with AP.
def clustering(dir, output_dir, model, dataset):
    paths = os.listdir(dir)
    files = sorted([file for file in paths if file.endswith(".csv")])

    ap_res = {}
    # output = {}
    for file in files:
        df = pd.read_csv(os.path.join(dir, file))

        if "Unnamed: 0" in df.columns:
            df.drop(df.columns[0], axis=1, inplace=True)
            df.set_index(df.columns[0], inplace=True)

        else:
            df.set_index(df.columns[0], inplace=True)

        df = df.fillna(df.median())

        # AP clustering.
        model.fit(df)
        stations = list(model.cluster_centers_indices_)

        indices= {}
        for index, station in enumerate(list(stations)):
            indices[index+1] = station

        # assingning cluster labels to the stations
        labels = model.labels_

        # new_labels = labels + 1

        # print(new_labels)
        # print(indices)

        # labels = labels + 1
     
        # labels = [indices[station] for station in labels]


        # ap_res[file.split("/")[-1].split(".")[0]] = list(labels)
        # result = pd.DataFrame(ap_res).set_index(df.index)

    # result.to_csv(os.path.join(output_dir, f'{dataset}.csv'), index=True)
    # return result

# riverflow metrics clustering
# datasets = ["func", "func_climate", "all_attributes"]
datasets = ["func"]

for dataset in datasets:
    dir = os.path.join(directory, "computed_data/pca_results", dataset)
    outputdir = os.path.join(directory, "computed_data/ap_results")
    res = clustering(dir, outputdir, model, dataset)
    # print(res)

In [ ]:
# check the number of clusters
dir = os.path.join(directory, 'computed_data/ap_results')
paths = os.listdir(dir)
files = sorted([file for file in paths if file.endswith(".csv")])

ap_results = {}
for file in files:
    df = pd.read_csv(os.path.join(dir, file))
    for col in df.columns:
        if col == 'station_id':
            continue
        ap_results[col] = df[col].nunique()

# ap_results